In [386]:
%run "get_packages.py"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from pickle import TRUE
from dateutil.relativedelta import relativedelta
import os
from skopt import BayesSearchCV  # need scikit-optimize
from skopt.space import Real, Integer
import xbbg
import blp
from sklearn import preprocessing
import openpyxl
import plotly.express as px
import statsmodels
from xbbg import blp
import datetime
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import BDay

In [387]:
today = date.today()
end_date = today.strftime("%Y%m%d")
strt_date="20241030"
std_size = 0.03
concentration_size = 0.05
step_size = 0.005
TE_upper_bound = 0.012
TE_lower_bound = 0.008
features_weight = 0.33
multiplier = 2
carry_cost_lim = 0.003
#start_date = 1/1/2003

REER_lag = 22
TB_lag = 30
GDP_lag = 120
CPI_lag = 30

Adj_REER = ["REER", "Conditional REER", "Symmetrical REER"]
Carry = ["Nominal Carry", "Real Carry"]
Conviction = ["Awa", "Agn"]

currencies = ["BRL", "CLP", "CNY", "COP", "CZK", "HUF", "IDR", "MYR", "MXN", "PEN", "PHP", "PLN", "RON", "RUB", "ZAR", "THB", "TRY", "INR", "SGD", "TWD", "KRW"]


In [388]:
#SPOT PRICE
from blp import blp
bquery = blp.BlpQuery().start()
spot_query = bquery.bdh(
        ["BRL BGN Curncy", "CLP BGN Curncy", "CNY CMPN Curncy", "COP REGN Curncy", "CZK BGN Curncy", "HUF BGN Curncy", "IDR CMPN Curncy", "MYR CMPN Curncy", "MXN BGN Curncy", "PEN BGN Curncy", "PHP CMPN Curncy", "PLN BGN Curncy", "RON BGN Curncy", "RUB BGN Curncy", "ZAR BGN Curncy", "THB BGN Curncy", "TRY BGN Curncy", "INR CMPN Curncy", "SGD BGN Curncy", "TWD BGN Curncy", "KRW CMPN Curncy"],
        ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})

spot_query['security_short'] = spot_query['security'].str[:3]

spot = spot_query.pivot_table(index='date', columns='security_short', values='PX_LAST', aggfunc='first')
spot.reset_index( inplace=True)
spot.rename(columns={'date':'DATE'}, inplace=True)
spot['DATE'] = pd.to_datetime(spot['DATE'])
spot.sort_values(by='DATE', ascending=False, inplace=True)
spot.dropna(inplace=True)

# Extract relevant columns
columns = ["DATE"] + currencies
spot_subset = spot[columns]

# Calculate daily returns
spot_subset.set_index("DATE", inplace=True)
daily_return = spot_subset.shift(-1)/spot_subset  -1
daily_return.reset_index(inplace=True)
#daily_return.dropna(inplace=True)
spot_return = daily_return.melt(id_vars=["DATE"], var_name="Currency", value_name="Daily_Return")


In [389]:
bquery = blp.BlpQuery().start()
MA_query = bquery.bdh(
        ["BRL Curncy","CLP Curncy","CNY Curncy","COP Curncy","CZK Curncy","HUF Curncy","IDR Curncy","MYR Curncy","MXN Curncy","PEN Curncy","PHP Curncy","PLN Curncy","RON Curncy","RUB Curncy","ZAR Curncy","THB Curncy","TRY Curncy","INR Curncy","SGD Curncy","TWD Curncy","KRW Curncy"],
        ["MOV_AVG_5D","MOV_AVG_20D"],
        start_date=strt_date,
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})
MA_query.rename(columns={'date':'DATE'}, inplace=True)
MA_query['Currency'] = MA_query['security'].str[:3]
MA_query.drop(columns="security", inplace=True)
MA_query['Momentum'] = np.where(MA_query['MOV_AVG_5D'] > MA_query['MOV_AVG_20D'], 1, -1)
Momentum = MA_query[["DATE", "Currency", "MOV_AVG_5D", "MOV_AVG_20D","Momentum"]]
Momentum['DATE'] = pd.to_datetime(Momentum['DATE'])
Momentum.sort_values(by='DATE', ascending=False, inplace=True)
Momentum.reset_index(drop=True, inplace=True)
#Momentum.dropna(inplace=True)


C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pandas.concat(dfs)


,DATE,Currency,MOV_AVG_5D,MOV_AVG_20D,Momentum
0,2024-12-06,THB,34.3190,34.5540,-1
1,2024-12-06,CZK,23.8813,23.9580,-1
2,2024-12-06,TRY,34.7416,34.5638,1
3,2024-12-06,SGD,1.3429,1.3423,1
4,2024-12-06,TWD,32.4850,32.4890,-1
...,...,...,...,...,...
583,2024-10-30,TRY,34.2738,34.2391,1
584,2024-10-30,MYR,4.3619,4.3096,1
585,2024-10-30,KRW,1384.0800,1363.4000,1
586,2024-10-30,INR,84.0798,84.0379,1


In [390]:
def create_momentum_dict(momentum_df):
    """Creates a dictionary of DataFrames with momentum data organized by date.

    Args:
        momentum_df: The input DataFrame with 'DATE', 'Currency', 'MOV_AVG_5D', 'MOV_AVG_20D', and 'Momentum' columns.

    Returns:
        A dictionary where keys are dates (YYYYMMDD strings) and values are DataFrames
        with 'Currency' as index and 'MOV_AVG_5D', 'MOV_AVG_20D', and 'Momentum' as columns.
    """

    momentum_dict = {}
    for date, date_df in momentum_df.groupby('DATE'):
        date_str = date.strftime('%Y%m%d')
        
        if date_str not in momentum_dict: #Create empty dataframes for dates to avoid errors if data is missing.
            momentum_dict[date_str] = pd.DataFrame(index = currencies, columns=['MOV_AVG_5D', 'MOV_AVG_20D', 'Momentum'])



        temp_df = date_df.set_index('Currency')


        temp_df = temp_df.reindex(currencies)




        momentum_dict[date_str] = temp_df #


    return momentum_dict


momentum_dict = create_momentum_dict(Momentum) #

In [391]:
#3M Implied vol
from blp import blp 
bquery = blp.BlpQuery().start()
IMPVOL_query = bquery.bdh(
        ["USDBRLV3M Curncy","USDCLPV3M Curncy","USDCNYV3M Curncy","USDCOPV3M Curncy","USDCZKV3M Curncy","USDHUFV3M Curncy","USDIDRV3M Curncy","USDMYRV3M Curncy","USDMXNV3M Curncy","USDPENV3M Curncy","USDPHPV3M Curncy","USDPLNV3M Curncy","USDRONV3M Curncy","USDRUBV3M Curncy","USDZARV3M Curncy","USDTHBV3M Curncy","USDTRYV3M Curncy","USDINRV3M Curncy","USDSGDV3M Curncy","USDTWDV3M Curncy","USDKRWV3M Curncy"],
        ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"}) #

IMPVOL_query.rename(columns={'date':'DATE', 'PX_LAST':'3M Implied Vol'}, inplace=True)

conditions = [
    (IMPVOL_query['security'] == 'USDBRLV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCLPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCNYV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCOPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCZKV3M Curncy'),
    (IMPVOL_query['security'] == 'USDHUFV3M Curncy'),
    (IMPVOL_query['security'] == 'USDIDRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDMYRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDMXNV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPENV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPHPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPLNV3M Curncy'),
    (IMPVOL_query['security'] == 'USDRONV3M Curncy'),
    (IMPVOL_query['security'] == 'USDRUBV3M Curncy'),
    (IMPVOL_query['security'] == 'USDZARV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTHBV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTRYV3M Curncy'),
    (IMPVOL_query['security'] == 'USDINRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDSGDV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTWDV3M Curncy'),
    (IMPVOL_query['security'] == 'USDKRWV3M Curncy'),
]

IMPVOL_query['Currency'] = np.select(conditions, currencies, default=None)
IMPVOL_query.drop(columns="security", inplace=True)
IMPVOL_query.sort_values(by='DATE', ascending=False, inplace=True)
IMPVOL_query=IMPVOL_query[['DATE','Currency','3M Implied Vol']]


In [392]:

def create_volatility_dict(volatility_df):
    """Creates a dictionary of DataFrames with volatility data organized by date.

    Args:
        volatility_df: Input DataFrame with 'DATE', 'Currency', and '3M Implied Vol' columns.

    Returns:
        A dictionary with dates ('YYYYMMDD') as keys and DataFrames as values.
        DataFrames have 'Currency' as index and '3M Implied Vol' as column.
    """

    volatility_dict = {}

    for date, date_df in volatility_df.groupby('DATE'):  # Group by date
        date_str = date.strftime('%Y%m%d')

        if date_str not in volatility_dict:
             volatility_dict[date_str] = pd.DataFrame(index=currencies, columns=['3M Implied Vol'])



        temp_df = date_df.set_index('Currency') # Set Currency as index

        temp_df = temp_df.reindex(currencies)  # Reindex to ensure all currencies and order


        volatility_dict[date_str] = temp_df


    return volatility_dict

volatility_dict = create_volatility_dict(IMPVOL_query)


In [393]:
import pandas as pd
from pandas.tseries.offsets import BDay
import numpy as np
from blp import blp 
all_currencies = ["BRL","CLP","CNY","COP","CZK","HUF","IDR","MYR","MXN","PEN","PHP","PLN","RON","RUB","ZAR","THB","TRY","INR","SGD","TWD","KRW","UAH","EGP","EUR","JPY","USD","GBP","CAD","ARS","CHF"]

# Create an empty list to store the dataframes
all_cpi_data = []
#bquery = blp.BlpQuery().start()
# Loop through the dates from 2020 to today
start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('today') 
for date in pd.date_range(start=start_date, end=end_date, freq='MS'): # 'MS' for month start frequency
    specific_date = date + pd.DateOffset(months=1) - pd.DateOffset(days=1)  # Get the last day of the month

    begin = ((specific_date - pd.DateOffset(months=2)).replace(day=1) + pd.DateOffset(months=1) - pd.DateOffset(days=1) - BDay(1)).strftime('%Y%m%d')
    end = ((specific_date - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=1) - pd.DateOffset(days=1) - BDay(1)).strftime('%Y%m%d')

    bquery = blp.BlpQuery().start()
    CPI_query = bquery.bdh(
            ["BZPIIPCM Index","CLINNSMO Index","CNCPIMOM Index","COCPIMOM Index","CZCPMOM Index","HUCPIMM Index","IDCPIM Index","MACPIMOM Index","MXCPCHNG Index","PRCPNONC Index","PHC2MOM Index","POCPIMOM Index","ROCOPMOM Index","RUCPIMOM Index","SACPIMOM Index","THCPIMOM Index","TUCPIM Index","INFUTOT Index","SICPIMOM Index","TWCPIMOM Index","KOCPIMOM Index","UACPTMOM Index","EGCPMOM Index","ECCPEMUM Index","JNCPIMOM Index","CPURNSA% Index","UKRPCJMR Index","CACPICHG Index","ARNCNINX Index","SZCPIMOM Index"],
            ["PX_LAST"],
            start_date= begin,
            end_date= end, 
            options={"adjustmentSplit": True, "nonTradingDayFillOption":"ALL_CALENDAR_DAYS", "periodicitySelection":"MONTHLY", 'nonTradingDayFillMethod':'NIL_VALUE'})
    CPI_query.rename(columns={'PX_LAST':'CPI'}, inplace=True)

    conditions = [
    (CPI_query['security'] == 'BZPIIPCM Index'),
    (CPI_query['security'] == 'CLINNSMO Index'),
    (CPI_query['security'] == 'CNCPIMOM Index'),
    (CPI_query['security'] == 'COCPIMOM Index'),
    (CPI_query['security'] == 'CZCPMOM Index'),
    (CPI_query['security'] == 'HUCPIMM Index'),
    (CPI_query['security'] == 'IDCPIM Index'),
    (CPI_query['security'] == 'MACPIMOM Index'),
    (CPI_query['security'] == 'MXCPCHNG Index'),
    (CPI_query['security'] == 'PRCPNONC Index'),
    (CPI_query['security'] == 'PHC2MOM Index'),
    (CPI_query['security'] == 'POCPIMOM Index'),
    (CPI_query['security'] == 'ROCOPMOM Index'),
    (CPI_query['security'] == 'RUCPIMOM Index'),
    (CPI_query['security'] == 'SACPIMOM Index'),
    (CPI_query['security'] == 'THCPIMOM Index'),
    (CPI_query['security'] == 'TUCPIM Index'),
    (CPI_query['security'] == 'INFUTOT Index'),
    (CPI_query['security'] == 'SICPIMOM Index'),
    (CPI_query['security'] == 'TWCPIMOM Index'),
    (CPI_query['security'] == 'KOCPIMOM Index'),
    (CPI_query['security'] == 'UACPTMOM Index'),
    (CPI_query['security'] == 'EGCPMOM Index'),
    (CPI_query['security'] == 'ECCPEMUM Index'),
    (CPI_query['security'] == 'JNCPIMOM Index'),
    (CPI_query['security'] == 'CPURNSA% Index'),
    (CPI_query['security'] == 'UKRPCJMR Index'),
    (CPI_query['security'] == 'CACPICHG Index'),
    (CPI_query['security'] == 'ARNCNINX Index'),
    (CPI_query['security'] == 'SZCPIMOM Index')
    ]

    CPI_query['Currency'] = np.select(conditions, all_currencies, default=None)
    CPI_query.loc[CPI_query['Currency'] == 'INR', 'CPI'] = 100*(CPI_query['CPI']/185.8 -1)
        
    CPI_query=CPI_query[['Currency','CPI']]
    CPI_query['Date'] = specific_date # Add a Date column

    all_cpi_data.append(CPI_query) # Append the dataframe to the list


# Concatenate all dataframes vertically
final_cpi_data = pd.concat(all_cpi_data, ignore_index=True)

CPI_query = final_cpi_data.copy()

C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pandas.concat(dfs)
C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pandas.concat(dfs)
C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determinin

In [394]:
###########  extract CPI from this 
tickers = [
    "BZPIIPCY Index", "CLINSYOY Index", "CNCPIY0Y Index", "COCPIY0Y Index", "CZCPIY0Y Index", 
    "HUCPIY Index", "IDCPIY Index", "INCPIY Index", "MACPIY0Y Index", "MXCPIY0Y Index",
    "MYRCPYOY Index", "PRCPYOY Index", "PHCPIY Index", "PLCPIY0Y Index", "POCPIY0Y Index", 
    "ROCPIY0Y Index", "RUCPIY0Y Index", "SACPIYOY Index", "THCPIY0Y Index", "TUCPIY Index", 
    "TWCPIY0Y Index", "KOCPIY0Y Index", "UACPIY0Y Index", "EGCPIY0Y Index", "ECCPEGUY Index", 
    "JNCPIY0Y Index", "IW/FUTOTY Index", "SICPIY0Y Index"
]

cpi_tickers = pd.DataFrame({'Ticker': tickers})
cpi_value = CPI_query.loc[CPI_query['Currency'] == 'USD', 'CPI'].iloc[0]

from blp import blp 
ticker_to_currency = {
    "BZPIIPCY Index": "BRL",
    "CLINNSYO Index": "CLP",
    "CNCPIYOY Index": "CNY",
    "COCPIYOY Index": "COP",
    "CZCPYOY Index": "CZK",
    "HUCPIYY Index": "HUF",
    "IDCPIY Index": "IDR",
    "MACPIYOY Index": "CAD",
    "MXCPYOY Index": "MXN",
    "PRCPYOY Index": "PEN",
    "PHC2II Index": "PHP",
    'MACPIYOY Index': 'MYR',
    'SACPIYOY Index': 'ZAR',
    "POCPIYOY Index": "PLN",
    "ROCOPYOY Index": "RON",
    "RUCPIYOY Index": "RUB",
    "SICPIYOY Index": "SGD",
    "THCPIYOY Index": "THB",
    "TUCPIY Index": "TRY",
    "INFUTOTY Index": "INR",
    "TWCPIYOY Index": "TWD",
    "KOCPIYOY Index": "KRW",
    "UACPTYOY Index": "UAH",
    "EGCPYOY Index": "EGP",
    "ECCPEMUY Index": "EUR",
    "JNCPIYOY Index": "JPY",
    "CPI YOY Index": "USD"  # Assuming this is the US CPI YOY index
}

# Create an empty list to store the data
data = []

# Start a Bloomberg session

# Iterate through the tickers and retrieve the last price
for ticker, currency in ticker_to_currency.items():
    try:
        # Retrieve the last price
        last_price_df = bquery.bdh([ticker], ["last_price"],
            start_date= '20241001',
            end_date = today.strftime("%Y%m%d"), 
            options={"adjustmentSplit": True, "nonTradingDayFillOption":"ALL_CALENDAR_DAYS", "periodicitySelection":"MONTHLY", 'nonTradingDayFillMethod':'NIL_VALUE'})
        last_price = last_price_df['last_price'].iloc[0] 

        # Append the data to the list
        data.append([currency, last_price])
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")

# Create the DataFrame from the collected data
inflation  = pd.DataFrame(data, columns=['Currency', 'CPI YOY'])

In [395]:

def create_cpi_dict(final_cpi_data, all_currencies):
    cpi_dict = {}
    final_cpi_data['Date'] = pd.to_datetime(final_cpi_data['Date'])
    for date in final_cpi_data['Date'].unique():
        date_str = date.strftime('%Y-%m-%d')  # Format date as string for consistency
        date_data = final_cpi_data[final_cpi_data['Date'] == date].copy()

        # Create the correctly formatted DataFrame
        temp_df = pd.DataFrame(index=all_currencies, columns=['CPI']) # Use all_currencies as index
        for currency in all_currencies:  
            if (date_data['Currency'] == currency).any():  # Check if the currency exists for this date
                temp_df.loc[currency, 'CPI'] = date_data.loc[date_data['Currency'] == currency, 'CPI'].iloc[0]
            else:
                temp_df.loc[currency, 'CPI'] = np.nan  # Handle missing currencies
        temp_df = temp_df.reset_index().rename(columns={'index':'Currency'}) 
        cpi_dict[date_str] = temp_df  # Store with date string as key

    return cpi_dict
all_currencies = ["BRL","CLP","CNY","COP","CZK","HUF","IDR","MYR","MXN","PEN","PHP","PLN","RON","RUB","ZAR","THB","TRY","INR","SGD","TWD","KRW","UAH","EGP","EUR","JPY","USD","GBP","CAD","ARS","CHF"]
cpi_dict = create_cpi_dict(final_cpi_data, all_currencies)

In [396]:
from xbbg import blp

def get_reer_data(date, securities, currencies):
    begin = ((date - pd.DateOffset(months=150)).replace(day=1)).strftime('%Y-%m-%d')
    end = date.strftime('%Y-%m-%d')

    REER_query = blp.bdh(
        securities,
        ["PX_LAST"],
        start_date=begin,
        end_date=end
    )

    REER_query.columns = REER_query.columns.get_level_values(0)  # Flatten MultiIndex

    ticker_to_currency = dict(zip(securities, currencies))
    REER_query = REER_query.rename(columns=ticker_to_currency)  #Rename currency tickers

    REER_query = REER_query.stack().reset_index() #Stacking to multiindex
    REER_query.columns = ['date', 'Currency', 'PX_LAST'] #Renaming accordingly because of stack's naming convention for the tickers columns
   
    REER_query = REER_query.pivot(index='date', columns='Currency', values='PX_LAST') #pivoting correctly after having stacked

    return REER_query


# Example Usage (replace with your lists and date)
securities = ["BISBBRR Index","BISBCLR Index","BISBCNR Index","BISBCOP Index","BISBCZR Index","BISBHUR Index","BISBIDR Index","BISBMYR Index","BISBMXR Index","BISBPER Index","BISBPHR Index","BISBPLR Index","BISBROR Index","BISBRUR Index","BISBZAR Index","BISBTHR Index","BISBTRR Index","BISBINR Index","BISBSGR Index","BISBTWR Index","BISBKRR Index"]
currencies = ["BRL", "CLP", "CNY", "COP", "CZK", "HUF", "IDR", "MYR", "MXN", "PEN", "PHP", "PLN", "RON", "RUB", "ZAR", "THB", "TRY", "INR", "SGD", "TWD", "KRW"]
date = pd.to_datetime('2024-11-26')

reer_data = get_reer_data(date, securities, currencies)
reer_data = reer_data.reset_index(drop=False)  # Remove current index if date isn't a column


In [397]:

def calculate_5yr_avg_reer(reer_data, currencies):
    reer_data['date'] = pd.to_datetime(reer_data['date'])
    reer_data = reer_data.sort_values('date')
    reer_data = reer_data.set_index('date')

    for currency in currencies:
        reer_data[f'{currency}_5yr_rolling'] = np.nan  # Initialize

        for i in range(len(reer_data)):
            current_date = reer_data.index[i]
            five_years_ago = current_date - pd.DateOffset(months=61) #Modified to start 61 months ago
            past_month = current_date - pd.DateOffset(months=1) #Defined past month
            past_data = reer_data.loc[five_years_ago:past_month, currency].dropna() #Selects data between 61 months ago and last month (excluded), removing NaNs

            if len(past_data) >= 60:
                reer_data.loc[current_date, f'{currency}_5yr_rolling'] = past_data.mean()

    return reer_data.reset_index(drop=False)

securities = ["BISBBRR Index", "BISBCLR Index", "BISBCNR Index",  "BISBCOP Index","BISBCZR Index", "BISBHUR Index", "BISBIDR Index", "BISBMYR Index", "BISBMXR Index",  "BISBPER Index","BISBPHR Index",  "BISBPLR Index",  "BISBROR Index",  "BISBRUR Index","BISBZAR Index", "BISBTHR Index",  "BISBTRR Index", "BISBINR Index", "BISBSGR Index", "BISBTWR Index", "BISBKRR Index"] #Use actual tickers
currencies = ['BRL', 'CLP', 'CNY', 'COP', 'CZK', 'HUF', 'IDR','INR','KRW','MYR','MXN','PEN','PHP','PLN','RON','RUB','SGD','THB','TRY','TWD','ZAR']# Actual list matching ticker elements without the date column.
date = pd.to_datetime('2024-09-30') # Your target date
reer_data = calculate_5yr_avg_reer(reer_data, currencies)

# Convert the 'date' column to datetime objects if it's not already
reer_data['date'] = pd.to_datetime(reer_data['date'])

# Format the dates as 'YYYY-MM-DD' strings
reer_data['date'] = reer_data['date'].dt.strftime('%Y-%m-%d')


In [398]:
reer_dict = {}


for date in reer_data['date'].unique():
    date_data = reer_data[reer_data['date'] == date].copy()
    date_data = date_data.drop(columns='date')

    temp_df = pd.DataFrame(index=currencies, columns=['REER', 'REER_5Y_avg'])

    for currency in currencies:
        reer_col = currency
        rolling_col = currency + "_5yr_rolling"

        if rolling_col in reer_data.columns:
            temp_df.loc[currency, 'REER'] = date_data[reer_col].iloc[0]
            temp_df.loc[currency, 'REER_5Y_avg'] = date_data[rolling_col].iloc[0]
        else:
            temp_df.loc[currency, 'REER'] = date_data[reer_col].iloc[0]
            temp_df.loc[currency, 'REER_5Y_avg'] = np.nan

    # Add Currency as a column before resetting indexFTB_3
    
    temp_df = temp_df.reset_index().rename(columns={'index': 'Currency'}) 
    reer_dict[date] = temp_df

In [399]:
# trading weights is an independent part 

from blp import blp 
weights = {'BRL': { 'EUR':19.8326891433532, 'CNY': 19.31908920559, 'USD': 18.9827326781795, 'ARS': 9.04088735496284, 'KRW': 4.00326543003275}, 
         'CLP': { 'CNY':32.4903579496702, 'USD': 15.7403934634422, 'EUR': 15.7221929477127, 'BRL': 5.30539472417201, 'KRW': 4.85057074105527},
         'CNY': { 'USD':19.6807605546305, 'EUR': 18.034148790699, 'JPY': 11.9337085872547, 'KRW': 8.77690798092872, 'TWD': 5.94072168056887},
         'COP': { 'CNY':23.7228279745679, 'USD': 22.3958427765579, 'EUR': 15.0561850379673, 'MXN': 9.47818212291651, 'BRL': 5.79804369588218},
         'CZK': { 'EUR':53.7357432005846, 'CNY': 9.32263987744015, 'PLN': 6.48343861856898, 'USD': 4.57420251197199, 'GBP': 4.46219657045916},
         'HUF': { 'EUR':51.4462139331452, 'CNY': 8.5469463341168, 'USD': 5.49254146539307, 'PLN': 4.89862553996062, 'CZK': 4.01613062014827},
         'IDR': { 'CNY':23.2668581166172, 'JPY': 12.4015844911697, 'EUR': 10.7498551165224, 'SGD': 9.67219382822659, 'USD': 9.16342505380052},
         'MYR': { 'CNY':23.7465947578572, 'USD': 13.0350647634701, 'EUR': 11.4157257777885, 'JPY': 9.45471669844536, 'SGD': 8.43320915760798},
         'MXN': { 'USD':53.7852968728671, 'CNY': 15.3570822350254, 'EUR': 8.85721047352065, 'JPY': 3.68473624173183, 'CAD': 3.65211347020733},
         'PEN': { 'CNY':27.8771478535743, 'USD': 18.0329542150098, 'EUR': 12.372696604013, 'BRL': 6.19816245225753, 'MXN': 5.04137616220383},
         'PHP': { 'CNY':20.860353198127, 'JPY': 15.316588250294, 'USD': 12.1367056047445, 'EUR': 10.5339870265713, 'KRW': 6.90886495923643},
         'PLN': { 'EUR':51.2744348107249, 'CNY': 11.8457719861224, 'GBP': 4.84680781974116, 'USD': 4.56041842863057, 'CZK': 3.76300172956997},
         'RON': { 'EUR':53.163229044445, 'CNY': 7.52963234306008, 'HUF': 4.58419055313407, 'PLN': 4.30552373422745, 'GBP': 4.06001591246192},
         'RUB': { 'EUR':36.5454149814027, 'CNY': 21.6181286031918, 'USD': 7.13934430006148, 'JPY': 4.60563918801572, 'KRW': 3.55071673784711},
         'ZAR': { 'EUR':28.0706716305097, 'CNY': 21.9557456326697, 'USD': 11.4296519508831, 'JPY': 6.34943112088636, 'GBP': 4.6456625826538},
         'THB': { 'CNY':23.5940786001256, 'JPY': 15.7028546336318, 'USD': 10.731028129059, 'EUR': 10.454098946585, 'MYR': 4.48452665287658},
         'TRY': { 'EUR':40.3579265739337, 'CNY': 14.6056217841577, 'USD': 6.35823987754735, 'GBP': 5.14480031288076, 'KRW': 3.5563885504494},
         'INR': { 'CNY':23.0813819215815, 'EUR': 18.4229102981809, 'USD': 13.9630590579442, 'KRW': 4.68345461835726, 'JPY': 4.55859195195944},
         'SGD': { 'CNY':22.4672690983772, 'EUR': 11.4224275576858, 'USD': 11.3383351154703, 'JPY': 7.95992633793645, 'MYR': 7.31117597215136},
         'TWD': { 'CNY':34.561520327848, 'JPY': 13.5244959365383, 'USD': 12.9747862420664, 'EUR': 10.1631302332482, 'KRW': 5.92150265900429},
         'KRW': { 'CNY':33.2674159298525, 'USD': 13.9679646593416, 'EUR': 12.7550195378573, 'JPY': 10.8782446505293, 'TWD': 4.03210645129297},
         'EUR': { 'CNY':18.3714673852213, 'USD': 14.5632639529848, 'GBP': 10.440394166651, 'CHF': 5.54635594069113, 'PLN': 5.50799014172826},
         'JPY': { 'CNY':31.8896059983949, 'USD': 16.4932499426125, 'EUR': 12.6796695252115, 'KRW': 5.95597200870829, 'TWD': 4.60760731009828}
}

df = pd.DataFrame.from_dict(weights, orient='index').fillna(0)
df['total_weight'] = df.apply(sum, axis =1)
df.reset_index(inplace=True)
df.rename(columns={'index':'Currency'}, inplace=True)
trading_weights = pd.melt(df,id_vars=["Currency", 'total_weight'], var_name="Trading partner", value_name="weight")
trading_weights = trading_weights[trading_weights['weight'] != 0]


In [400]:
########## DATA PRICES 
from xbbg import blp 
import pandas as pd
from dateutil.relativedelta import relativedelta

# Define the list of Bloomberg tickers
tickers = ["BRL BGN Curncy", "CLP BGN Curncy", "CNY CMPN Curncy", "COP REGN Curncy",
           "CZK BGN Curncy", "HUF BGN Curncy", "IDR CMPN Curncy", "MYR CMPN Curncy",
           "MXN BGN Curncy", "PEN BGN Curncy", "PHP CMPN Curncy", "PLN BGN Curncy",
           "RON BGN Curncy", "RUB BGN Curncy", "ZAR BGN Curncy", "THB BGN Curncy",
           "TRY BGN Curncy", "INR CMPN Curncy", "SGD BGN Curncy", "TWD BGN Curncy",
           "KRW CMPN Curncy"]

# Extract currency codes from tickers
currencies = [ticker[:3] for ticker in tickers]

# Get historical data for the past 5 years
end_date = pd.Timestamp.today()
start_date = end_date - relativedelta(years=5)

data_prices = blp.bdh(tickers, 'PX_LAST', '20150101', end_date)
data_prices.columns = currencies
data_prices


all_business_dates = pd.to_datetime(data_prices.index)
all_business_dates_str = [date.strftime('%Y-%m-%d') for date in all_business_dates]

all_business_dates_str[-50:]
all_business_dates = pd.to_datetime(data_prices.index)


In [401]:
from xbbg import blp 
import pandas as pd
from dateutil.relativedelta import relativedelta

# Define the list of Bloomberg tickers
tickers = ["BRL BGN Curncy", "CLP BGN Curncy", "CNY CMPN Curncy", "COP REGN Curncy",
           "CZK BGN Curncy", "HUF BGN Curncy", "IDR CMPN Curncy", "MYR CMPN Curncy",
           "MXN BGN Curncy", "PEN BGN Curncy", "PHP CMPN Curncy", "PLN BGN Curncy",
           "RON BGN Curncy", "RUB BGN Curncy", "ZAR BGN Curncy", "THB BGN Curncy",
           "TRY BGN Curncy", "INR CMPN Curncy", "SGD BGN Curncy", "TWD BGN Curncy",
           "KRW CMPN Curncy"]

# Extract currency codes from tickers
currencies = [ticker[:3] for ticker in tickers]

# Get historical data for the past 5 years
end_date = pd.Timestamp.today()
start_date = end_date - relativedelta(years=5)

data_prices = blp.bdh(tickers, 'PX_MID', '20150101', end_date)
data_prices.columns = currencies
data_prices

,BRL,CLP,CNY,COP,CZK,HUF,IDR,MYR,MXN,PEN,...,PLN,RON,RUB,ZAR,THB,TRY,INR,SGD,TWD,KRW
2015-01-01,NaN,606.600,6.2077,NaN,22.88595,261.460,12405.0,3.4993,14.74675,NaN,...,3.54160,3.70435,57.80176,11.54804,32.9500,2.33138,63.22501,1.32351,31.6350,1093.245
2015-01-02,2.69420,613.413,6.2063,2374.000,23.05705,264.810,12526.0,3.5293,14.83840,2.97520,...,3.58623,3.73630,58.75000,11.68320,32.9700,2.34495,63.23451,1.33129,31.7730,1107.050
2015-01-05,2.70595,617.000,6.2219,2429.600,23.20675,266.393,12621.8,3.5570,14.94527,2.98950,...,3.59485,3.76720,61.61680,11.70768,32.9700,2.33056,63.48601,1.33533,31.9765,1109.645
2015-01-06,2.70075,616.625,6.2154,2450.170,23.26683,268.883,12652.4,3.5637,14.89475,2.98950,...,3.63188,3.78303,63.09165,11.71350,32.8600,2.32251,63.46051,1.33330,31.9900,1099.250
2015-01-07,2.68288,615.200,6.2116,2427.925,23.53110,268.470,12680.0,3.5709,14.71500,2.99050,...,3.62664,3.79890,62.90001,11.68082,32.8700,2.31947,63.15501,1.33972,31.9790,1098.740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-02,6.06810,978.205,7.2740,4460.080,24.04051,394.830,15896.5,4.4600,20.39821,3.75550,...,4.08120,4.74038,106.50000,18.13337,34.5004,34.71974,84.72506,1.34488,32.6310,1405.425
2024-12-03,6.05055,972.250,7.2882,4440.813,23.94380,394.375,15953.0,4.4700,20.31532,3.73760,...,4.09015,4.73585,105.00000,18.11590,34.3474,34.73285,84.68241,1.34485,32.5496,1415.500
2024-12-04,6.03979,974.955,7.2652,4415.730,23.94567,393.346,15897.0,4.4550,20.29653,3.73850,...,4.07220,4.73497,105.00000,18.16400,34.2738,34.74456,84.70776,1.34387,32.4305,1413.465
2024-12-05,6.00696,970.250,7.2597,4415.268,23.69062,390.045,15846.0,4.4280,20.19555,3.72668,...,4.03267,4.70388,100.75437,18.01566,34.1360,34.71364,84.67166,1.33893,32.4020,1415.240


In [403]:
def complete_reer_dict(reer_dict, all_business_dates_str):
    """Forward fills data within each month in the reer_dict.

    Preserves all existing keys in reer_dict. Processes dates from 2024 onwards.
    GUARANTEES no key removal.
    """
    reer_dict_completed = reer_dict.copy()
    all_business_dates = pd.to_datetime(all_business_dates_str).sort_values()
    all_business_dates_from_2024 = all_business_dates[all_business_dates >= pd.to_datetime('2024-01-01')]

    # Keep track of the latest available date *from any point in the data*
    last_available_date_overall = None
    for date_str in reer_dict:  # Pre-populate with existing dates from reer_dict if before 2024
        dt = pd.to_datetime(date_str)
        if last_available_date_overall is None or dt > last_available_date_overall:
            last_available_date_overall = dt


    for date in all_business_dates_from_2024:
        date_str = date.strftime('%Y-%m-%d')

        if date_str not in reer_dict_completed:

            previous_date = date - pd.DateOffset(days=1)
            previous_date_str = previous_date.strftime('%Y-%m-%d')



            while previous_date_str not in reer_dict_completed and previous_date >= all_business_dates[0]: # iterate from current date to earliest available date
                previous_date -= pd.DateOffset(days=1)
                previous_date_str = previous_date.strftime('%Y-%m-%d')


            if previous_date_str in reer_dict_completed:
                reer_dict_completed[date_str] = reer_dict_completed[previous_date_str].copy()

            # If no previous date exists in reer_dict_completed from 2024 onwards, check overall last_available_date_overall
            elif last_available_date_overall is not None:

                reer_dict_completed[date_str] = reer_dict[last_available_date_overall.strftime('%Y-%m-%d')].copy() # Use overall latest available date



            else:
                reer_dict_completed[date_str] = pd.DataFrame() # or preferred handling



        # Update last_available_date_overall AFTER checking and potentially filling a missing date.
        if date_str in reer_dict_completed:
            last_available_date_overall = date



    return reer_dict_completed
reer_dict2 = complete_reer_dict(reer_dict, all_business_dates_str)


In [404]:

def complete_cpi_dict(cpi_dict, all_business_dates_str):
    """Completes the cpi_dict by forward-filling missing dates with the last available data.

    Preserves all existing keys in cpi_dict.
    """
    cpi_dict_completed = cpi_dict.copy()
    all_business_dates = pd.to_datetime(all_business_dates_str).sort_values()

    last_available_date_str = None  # Keep track of the last date with available data
    last_available_value = None

    for date in all_business_dates:  # Iterate through all business dates
        date_str = date.strftime('%Y-%m-%d')

        if date_str in cpi_dict_completed: #Update the values for existing dates
            last_available_date_str = date_str
            last_available_value = cpi_dict_completed[date_str]
        elif last_available_value is not None : #If it is a missing date, fill with last available value
            cpi_dict_completed[date_str] = last_available_value

    return cpi_dict_completed

cpi_dict_2 = complete_cpi_dict(cpi_dict, all_business_dates_str)


In [405]:
cpi_dict_2 = complete_reer_dict(cpi_dict, all_business_dates_str)
#cpi_dict = cpi_dict_2.copy()

# reer_dict2 and cpi_dict2

In [406]:
LAST_UPDATE_query = '20241031' # enter the first date from which you construct the REER 
def get_adjusted_spot(row, date_current):  # Changed df to row
    """Calculates the adjusted spot."""
    currency = row['Currency']  # Access individual values, not Series
    trading_partner = row['Trading partner'] # Access individual values, not Series
    weight = row['weight']
    total_weight = row['total_weight']

    security_ticker = f"{currency}{trading_partner} Curncy"  # Correct ticker format


    try:  # Important: Handle potential Bloomberg errors!
        spot_tday = bquery.bdh([security_ticker], ["PX_LAST"], start_date=date_current, end_date=date_current, options={"adjustmentSplit": True}).loc[0, 'PX_LAST']
        spot_last = bquery.bdh([security_ticker], ["PX_LAST"], start_date=LAST_UPDATE_query, end_date=LAST_UPDATE_query, options={"adjustmentSplit": True}).loc[0, 'PX_LAST']


        return ((spot_tday / spot_last) - 1) * weight / total_weight

    except Exception as e:  # Handle any Bloomberg API or data errors
        print(f"Error retrieving data for {security_ticker}: {e}")
        return np.nan  # Return NaN for missing or incorrect data

# date_current is today 
date_current = today.strftime('%Y%m%d')
trading_weights['spot_adj'] = trading_weights.apply(get_adjusted_spot, date_current='20241126', axis=1) #date as string

Error retrieving data for MYRUSD Curncy: 0
Error retrieving data for TWDUSD Curncy: 0


In [407]:
from blp import blp 
def calculate_reer_discount(reer_dict2, trading_weights, cpi_dict_2, all_business_dates_str ):
    start_date_str = "2024-11-18"

# Create a filtered list of date strings
    filtered_dates = [date_str for date_str in all_business_dates_str if pd.to_datetime(date_str) > pd.to_datetime(start_date_str)]


    bquery = blp.BlpQuery().start() #Bloomberg query object
    for date_str in filtered_dates: #Iterate through all business dates (string format)
        
        if date_str in reer_dict2:
            print(date_str)
            #CPI_query = cpi_dict_2[date_str]
            #print(CPI_query)
            date = pd.to_datetime(date_str)
            last_day_of_previous_month = (date - pd.DateOffset(months=1) + pd.tseries.offsets.MonthEnd(0)).strftime('%Y%m%d')
            LAST_UPDATE_query = last_day_of_previous_month
            print(LAST_UPDATE_query)
            endofmonth = ((date- pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1))
            today = pd.Timestamp('today')

            date_query = date.strftime('%Y%m%d')

            # ADJUSTED SPOT (No changes needed, uses date and LAST_UPDATE_query)
            trading_weights['spot_adj'] = trading_weights.apply(get_adjusted_spot, date_current=date_query, axis=1)
              #Apply  get_adjusted spot with correct date variables

            # ADJUSTED INFLATION (Use date_str to access cpi_dict)
            try: #Try/except to handle KeyError if a date is missing
                # cpi_dict_2 is completed 
                print(type(date_str))
                CPI_query = cpi_dict_2[date_str]
            except KeyError:
                print(f"CPI data not found for {date_str}. Skipping REER discount calculation.")
                continue  # Go to the next date


            add_ccycpi = pd.merge(trading_weights, CPI_query, on='Currency')
            add_ccycpi.rename(columns={'CPI':'CPI_ccy'}, inplace=True)
            add_tpcpi = pd.merge(add_ccycpi, CPI_query, left_on='Trading partner', right_on='Currency').drop(['Currency_y'], axis = 1)
            add_tpcpi.rename(columns={'Currency_x':'Currency', 'CPI':'CPI_tp'}, inplace=True)
            add_tpcpi['infla_adj']=((add_tpcpi['CPI_ccy']-add_tpcpi['CPI_tp'])/100)*(add_tpcpi['weight']/add_tpcpi['total_weight'])
            add_tpcpi=add_tpcpi[['Currency','spot_adj','infla_adj']]

            # REER DISCOUNT PROXY (No changes, uses today, LAST_UPDATE, endofmonth)
            REER_proxy = add_tpcpi.groupby('Currency').sum().reset_index()
            REER_proxy['proxy'] = (1 + REER_proxy['infla_adj'])**((today - pd.to_datetime(LAST_UPDATE_query)).days/(endofmonth - pd.to_datetime(LAST_UPDATE_query)).days) * (1+REER_proxy['spot_adj']) #Corrected: Convert LAST_UPDATE_query to datetime
                    
            reer_dict2[date_str] = pd.merge(reer_dict2[date_str], REER_proxy, on='Currency')
            reer_dict2[date_str]['REER_Discount']=(reer_dict2[date_str]['REER']*reer_dict2[date_str]['proxy']/reer_dict2[date_str]['REER_5Y_avg'])-1
            reer_dict2[date_str].sort_values(by='Currency', ascending=True, inplace=True)
            

            # Merge and update reer_dict (use date_str as key)
            #reer_dict2[date_str] = pd.merge(reer_dict2[date_str], REER_proxy[['Currency', 'proxy']], on='Currency', how='left')

            #reer_dict2[date_str] = reer_dict2[date_str].reset_index().rename(columns={'index':'Currency'})
            #reer_dict2[date_str] = reer_dict2[date_str].set_index('Currency')



    bquery.stop() #Stop Bloomberg query session
    return reer_dict2

In [408]:
reer_dict_test = reer_dict2.copy()
reer_dict_test = calculate_reer_discount(reer_dict_test, trading_weights, cpi_dict_2, all_business_dates_str )

2024-11-19
20241031
Error retrieving data for MYRUSD Curncy: 0
Error retrieving data for TWDUSD Curncy: 0
<class 'str'>
2024-11-20
20241031
Error retrieving data for INRCNY Curncy: 0
Error retrieving data for BRLUSD Curncy: 0
Error retrieving data for MYRUSD Curncy: 0
Error retrieving data for INRUSD Curncy: 0
Error retrieving data for TWDUSD Curncy: 0
Error retrieving data for INRKRW Curncy: 0
<class 'str'>
2024-11-21
20241031
Error retrieving data for MYRUSD Curncy: 0
Error retrieving data for TWDUSD Curncy: 0
<class 'str'>
2024-11-22
20241031
Error retrieving data for MYRUSD Curncy: 0
Error retrieving data for TWDUSD Curncy: 0
<class 'str'>
2024-11-25
20241031
Error retrieving data for MYRUSD Curncy: 0
Error retrieving data for TWDUSD Curncy: 0
<class 'str'>
2024-11-26
20241031
Error retrieving data for MYRUSD Curncy: 0
Error retrieving data for TWDUSD Curncy: 0
<class 'str'>
2024-11-27
20241031
Error retrieving data for IDRUSD Curncy: 0
Error retrieving data for MYRUSD Curncy: 0
Er

In [409]:
strt_date = '20241030'

In [419]:
end_date = today.strftime('%Y%m%d')
bquery = blp.BlpQuery().start()
fpa3m_query = bquery.bdh(
        ["BCN3M BGN Curncy","CHN3M BGN Curncy","CNH3M CMPN Curncy","CLN3M BGN Curncy","CZK3M BGN Curncy","HUF3M BGN Curncy","IHN3M CMPN Curncy","MRN3M CMPN Curncy","MXN3M BGN Curncy","PSN3M BGN Curncy","PPN3M CMPN Curncy","PLN3M BGN Curncy","RON3M BGN Curncy","RUB3M BGN Curncy","ZAR3M BGN Curncy","THB3M BGN Curncy","TRY3M BGN Curncy","IRN3M CMPN Curncy","SGD3M BGN Curncy","NTN3M BGN Curncy","KWN3M CMPN Curncy"],
        ["PX_MID"],
        start_date=strt_date,
    end_date=end_date,
    options={"adjustmentSplit": True, "CDR": "5D"} )

fpa3m_query['Cntry_code'] = fpa3m_query['security'].str[:3]
fpa3m_query.rename(columns={'PX_MID':'FPA3M_MID'}, inplace=True)


In [420]:

conditions = [
    (fpa3m_query['Cntry_code'] == "BCN"),
    (fpa3m_query['Cntry_code'] == "CHN"),
    (fpa3m_query['Cntry_code'] == "CNH"),
    (fpa3m_query['Cntry_code'] == "CLN"),
    (fpa3m_query['Cntry_code'] == "CZK"),
    (fpa3m_query['Cntry_code'] == "HUF"),
    (fpa3m_query['Cntry_code'] == "IHN"),
    (fpa3m_query['Cntry_code'] == "MRN"),
    (fpa3m_query['Cntry_code'] == "MXN"),
    (fpa3m_query['Cntry_code'] == "PSN"),
    (fpa3m_query['Cntry_code'] == "PPN"),
    (fpa3m_query['Cntry_code'] == "PLN"),
    (fpa3m_query['Cntry_code'] == "RON"),
    (fpa3m_query['Cntry_code'] == "RUB"),
    (fpa3m_query['Cntry_code'] == "ZAR"),
    (fpa3m_query['Cntry_code'] == "THB"),
    (fpa3m_query['Cntry_code'] == "TRY"),
    (fpa3m_query['Cntry_code'] == "IRN"),
    (fpa3m_query['Cntry_code'] == "SGD"),
    (fpa3m_query['Cntry_code'] == "NTN"),
    (fpa3m_query['Cntry_code'] == "KWN")
]

# Assuming 'currencies' and 'fwd_scale_dict' are defined elsewhere in your code

fpa3m_query['Currency'] = np.select(conditions, currencies, default=None)
fpa3m_query.sort_values(by='date', ascending=True, inplace=True)
fpa3m_query.reset_index(inplace=True, drop=True)  # Add drop=True to avoid old index as a new column
fpa3m_query['FWD_SCALE'] = fpa3m_query['Currency'].map(fwd_scale_dict)
fpa3m_query['FPA3M_MID'] = fpa3m_query['FPA3M_MID'] / (10**fpa3m_query['FWD_SCALE'])



In [421]:

def create_fpa_dict(fpa_query):
    """Creates a dictionary of DataFrames with FPA data organized by date.

    Args:
        fpa_query: Input DataFrame with 'date_x', 'Currency', 'FPA3M_MID', and 'FPA1M_MID' columns.

    Returns:
        A dictionary with dates ('YYYYMMDD') as keys and DataFrames as values.
        DataFrames have 'Currency' as index and 'FPA3M_MID', 'FPA1M_MID' as columns.
        If a currency is missing for a date, its FPA values will be NaN.
    """

    fpa_dict = {}
    all_currencies = sorted(fpa_query['Currency'].unique())  # Get all unique currencies present


    for date, date_df in fpa_query.groupby('date'):  # Group by date
        date_str = date.strftime('%Y%m%d')

        # Create a DataFrame with all currencies as index for this date
        fpa_dict[date_str] = pd.DataFrame(index=all_currencies, columns=['FPA3M_MID'])


        temp_df = date_df.set_index('Currency')[['FPA3M_MID','date']] # Select only necessary columns

        # Update the DataFrame with the values from temp_df
        fpa_dict[date_str].update(temp_df)


    return fpa_dict

fpa_dict = create_fpa_dict(fpa3m_query)

In [422]:
from blp import blp 
end_date_str = pd.Timestamp(end_date).strftime('%Y%m%d') 
strt_date = '2018-01-01'
currencies = ["BRL", "CLP", "CNY", "COP", "CZK", "HUF", "IDR", "MYR", "MXN", "PEN", "PHP", "PLN", "RON", "RUB", "ZAR", "THB", "TRY", "INR", "SGD", "TWD", "KRW"]

bquery = blp.BlpQuery().start()

GDP_query = bquery.bdh(
        ["WGDPBRAZ Index","WGDPCHIL Index","WGDPCHIN Index","WGDPCOLO Index","WGDPCZEC Index","WGDPHUNG Index","WGDPINDO Index","WGDPMALY Index","WGDPMEXI Index","WGDPPERU Index","WGDPPHIL Index","WGDPPOLA Index","WGDPROMA Index","WGDPRUSS Index","WGDPSOUT Index","WGDPTHAI Index","WGDPTURK Index","WGDPINDI Index","WGDPSING Index","TAGDPARE Index","WGDPKORE Index"],
        ["PX_LAST"],
        start_date= '20210101', 
        end_date= end_date_str, 
        options={"adjustmentSplit": True, "nonTradingDayFillOption":"ALL_CALENDAR_DAYS", "periodicitySelection":"YEARLY", 'nonTradingDayFillMethod':'NIL_VALUE'})

GDP_query.rename(columns={'PX_LAST':'GDP'}, inplace=True)

conditions = [
    (GDP_query['security'] == 'WGDPBRAZ Index'),
    (GDP_query['security'] == 'WGDPCHIL Index'),
    (GDP_query['security'] == 'WGDPCHIN Index'),
    (GDP_query['security'] == 'WGDPCOLO Index'),
    (GDP_query['security'] == 'WGDPCZEC Index'),
    (GDP_query['security'] == 'WGDPHUNG Index'),
    (GDP_query['security'] == 'WGDPINDO Index'),
    (GDP_query['security'] == 'WGDPMALY Index'),
    (GDP_query['security'] == 'WGDPMEXI Index'),
    (GDP_query['security'] == 'WGDPPERU Index'),
    (GDP_query['security'] == 'WGDPPHIL Index'),
    (GDP_query['security'] == 'WGDPPOLA Index'),
    (GDP_query['security'] == 'WGDPROMA Index'),
    (GDP_query['security'] == 'WGDPRUSS Index'),
    (GDP_query['security'] == 'WGDPSOUT Index'),
    (GDP_query['security'] == 'WGDPTHAI Index'),
    (GDP_query['security'] == 'WGDPTURK Index'),
    (GDP_query['security'] == 'WGDPINDI Index'),
    (GDP_query['security'] == 'WGDPSING Index'),
    (GDP_query['security'] == 'TAGDPARE Index'),
    (GDP_query['security'] == 'WGDPKORE Index')
]

GDP_query['Currency'] = np.select(conditions, currencies, default=None)
GDP_query.sort_values(by='Currency', ascending=True, inplace=True)
GDP_query.reset_index(inplace=True)
GDP_query=GDP_query[['Currency','GDP']]


In [424]:
securities = ["WGDPBRAZ Index","WGDPCHIL Index","WGDPCHIN Index","WGDPCOLO Index","WGDPCZEC Index","WGDPHUNG Index","WGDPINDO Index","WGDPMALY Index","WGDPMEXI Index","WGDPPERU Index","WGDPPHIL Index","WGDPPOLA Index","WGDPROMA Index","WGDPRUSS Index","WGDPSOUT Index","WGDPTHAI Index","WGDPTURK Index","WGDPINDI Index","WGDPSING Index","TAGDPARE Index","WGDPKORE Index"]
fields = ["PX_LAST"]
start_date= '20180101'
end_date= end_date_str
options={"adjustmentSplit": True, "nonTradingDayFillOption":"ALL_CALENDAR_DAYS", "periodicitySelection":"YEARLY", 'nonTradingDayFillMethod':'NIL_VALUE'}

GDP_data = [] # list to store individual dataframes

for security in securities:
    try:
        df = bquery.bdh([security], fields, start_date=start_date, end_date=end_date_str, options=options)

        if not df.empty:
            df['Currency'] = currencies[securities.index(security)]  # Add currency using index
            GDP_data.append(df) # Append DataFrame
        else:
            print(f"No data retrieved for {security}")

    except Exception as e:
        print(f"Error retrieving data for {security}: {e}")

In [425]:
GDP_data = {}

for security in securities:
    try:
        df = bquery.bdh([security], fields, start_date=start_date, end_date=end_date_str, options=options)
        if not df.empty:
            currency = currencies[securities.index(security)]
            df.rename(columns={'PX_LAST': 'GDP'}, inplace=True)

            for index, row in df.iterrows():
                
                year = row['date'].year  # Get the year (which is now the index)
                print(year)
                date_str = str(year) + '-12-31'
  # Create date string YYYY1231 for end of year

                if date_str not in GDP_data:
                    GDP_data[date_str] = pd.DataFrame(columns=['Currency', 'GDP'])

                GDP_data[date_str] = pd.concat([GDP_data[date_str], pd.DataFrame({'Currency': [currency], 'GDP': [row['GDP']]})], ignore_index=True)
                

        else:
            print(f"No data retrieved for {security}")

    except Exception as e:
        print(f"Error retrieving data for {security}: {e}")


for date_str in GDP_data:
    GDP_data[date_str] = GDP_data[date_str].set_index('Currency')
    GDP_data[date_str] = GDP_data[date_str].reset_index()

2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023


C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_28404/1478019926.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  GDP_data[date_str] = pd.concat([GDP_data[date_str], pd.DataFrame({'Currency': [currency], 'GDP': [row['GDP']]})], ignore_index=True)
C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_28404/1478019926.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  GDP_data[date_str] = pd.concat([GDP_data[date_str], pd.DataFrame({'Currency': [currency], 'GDP': [row['GDP']]})], ignore_index=True)


2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023
2018
2019
2020
2021
2022
2023


In [426]:
from blp import blp 
bquery = blp.BlpQuery().start()
strt_date = '20230101'
end_date = pd.Timestamp('today').strftime('%Y%m%d')
###########################################################################################################################################################
###########################################################################################################################################################
bquery = blp.BlpQuery().start()
trade_balance_tickers = [
    "ECOYBBRN Index", "ECOYBCLN Index", "ECOYBCNN Index", "ECOYBCON Index",
    "ECOYBCZN Index", "ECOYBHUN Index", "ECOYBIDN Index", "ECOYBMYN Index",
    "ECOYBMXN Index", "PRTRBAL Index", "ECOYBPHN Index", "ECOYBPLN Index",
    "ECOYBRON Index", "ECOYBRUN Index", "ECOYBZAN Index", "ECOYBTHN Index",
    "ECOYBTRN Index", "ECOYBINN Index", "ECOYBSGN Index", "ECOYBTWN Index",
    "ECOYBKRN Index", "ECOYBEGN Index","ECOYBEGN Index", "ECOYBEAS Index", "ECOYBJPN Index" 
]

currencies = [
    "BRL", "CLP", "CNY", "COP", "CZK", "HUF", "IDR", "MYR", "MXN", "PEN",
    "PHP", "PLN", "RON", "RUB", "ZAR", "THB", "TRY", "INR", "SGD", "TWD",
    "KRW", "UAH", "EGP", "EUR", "JPY"
]

TB_query = bquery.bdh(trade_balance_tickers, ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})
TB_query.rename(columns={'PX_LAST':'TB'}, inplace=True)

# Create conditions dynamically
conditions = [(TB_query['security'] == ticker) for ticker in trade_balance_tickers]

TB_query['Currency'] = np.select(conditions, currencies, default=None) 

C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pandas.concat(dfs)


In [427]:

def create_tb_dict(tb_df):
    """Creates a dictionary of DataFrames with Trade Balance data organized by date.

    Args:
        tb_df: The input DataFrame with 'date', 'security', 'TB', and 'Currency' columns.

    Returns:
        A dictionary where keys are dates (YYYYMMDD strings) and values are DataFrames
        with 'Currency' as index and 'TB' as a column.
    """

    tb_dict = {}
    for date, date_df in tb_df.groupby('date'): #Group by date
        date_str = date.strftime('%Y%m%d')

        # Create a dictionary entry for the date if it doens't exist yet
        if date_str not in tb_dict:
            tb_dict[date_str] = pd.DataFrame()



        # Efficiently creates the DataFrame with "Currency" as index and "TB" as column:
        data = {currency: tb for currency, tb in zip(date_df['Currency'], date_df['TB'])} #uses zip to iterate through lists in parallel
        temp_df = pd.DataFrame.from_dict(data, orient='index', columns=['TB']) #orient = index to have currency as index
        temp_df = temp_df.reindex(currencies)
        temp_df = temp_df.reset_index().rename(columns={'index':'Currency'}) 

        tb_dict[date_str] = temp_df #add or update date in dictionary




    return tb_dict

In [428]:
tb_dict = create_tb_dict(TB_query)


In [429]:
def calculate_3m_tb_change(tb_dict):
    """Calculates the 3-month Trade Balance change for each DataFrame in the dictionary.

    Args:
        tb_dict: A dictionary where keys are dates ('YYYYMMDD') and values are DataFrames 
                 with 'Currency' as index and 'TB' as a column.

    Returns:
        A dictionary with the same structure as tb_dict, but with an additional 'TB_3m_Chg' column.
    """

    tb_dict_with_change = {}
    # treat trade balance PERU 
    
    for date_str, df in tb_dict.items():  # Iterate through the dictionary
        date = pd.to_datetime(date_str, format='%Y%m%d')  # Convert to datetime object

        # Calculate the date three months prior (using the last day of the month for alignment):
        three_months_ago = (date - pd.DateOffset(months=3)).strftime('%Y%m%d')


        if three_months_ago in tb_dict:  # Only calculate if data for three months ago exists


            df['TB_3m_Chg'] = df['TB'] - tb_dict[three_months_ago]['TB'] # Correct subtraction with aligned indices

        # Handle missing data if 3 months ago date isn't present (fill, 0, etc.)
        else:
            df['TB_3m_Chg'] = np.nan # Or your preferred handling


        tb_dict_with_change[date_str] = df  #Store modified DF in the dict


    return tb_dict_with_change

In [430]:
tb_dict_with_3m_change = calculate_3m_tb_change(tb_dict) #added this line


In [431]:

import blpapi
import pandas as pd
from datetime import date, timedelta

def get_settlement_dates(reference_date_str):
    """Retrieves settlement dates for the specified reference date."""

    request = refDataService.createRequest("ReferenceDataRequest")
    securities = [f"{currency}3M Curncy" for currency in currencies]
    for sec in securities:
        request.getElement("securities").appendValue(sec)
    request.getElement("fields").appendValue("SETTLE_DT")


    overrides = request.getElement("overrides")
    override1 = overrides.appendElement()
    override1.setElement("fieldId", "REFERENCE_DATE")
    override1.setElement("value", reference_date_str)


    session.sendRequest(request)
    results = {}

    while True:
        ev = session.nextEvent()
        if ev.eventType() == blpapi.Event.RESPONSE or ev.eventType() == blpapi.Event.PARTIAL_RESPONSE:
            for msg in ev:
                for securityData in msg.getElement("securityData"):
                    security = securityData.getElement("security").getValueAsString()
                    fieldData = securityData.getElement("fieldData")
                    if fieldData.numElements() > 0:
                        settle_dt = fieldData.getElement("SETTLE_DT").getValueAsString()
                        results[security] = settle_dt
                    else:
                        print(f"Warning: No SETTLE_DT data found for {security}.")
        if ev.eventType() == blpapi.Event.RESPONSE:
            break
            # ... (your existing code for processing the response)

    settle_dates_df = pd.DataFrame.from_dict(results, orient='index', columns=['Settlement_Date'])
    settle_dates_df.index.name = 'Security'
    return settle_dates_df



# Define date range (20 days ago to today)
today = date.today()
date_range = [today - timedelta(days=i) for i in range(50, -1, -1)]  # Includes today

settlement_dates_dict = {}

for d in date_range:
    date_str = d.strftime("%Y%m%d")
    try:
        settle_dates_df = get_settlement_dates(date_str)


        settle_dates_df.reset_index(inplace=True)
        # Check if CNY3M Curncy is missing and add it if necessary
        if 'CNY3M Curncy' not in settle_dates_df['Security'].values:
            new_row = {'Security': 'CNY3M Curncy', 'Settlement_Date': settle_dates_df['Settlement_Date'].iloc[0]} # Take first available settlement date
            settle_dates_df = pd.concat([settle_dates_df, pd.DataFrame(new_row, index=[0])])
            settle_dates_df.index = settle_dates_df.index.astype(str)
            settle_dates_df.sort_index(inplace=True)  # Optional sorting


        settlement_dates_dict[date_str] = settle_dates_df
    except Exception as e:
        print(f"Error retrieving data for {date_str}: {e}")



# --- Currency extraction and reordering OUTSIDE the loop ---
currency_order = ['BRL', 'CLP', 'CNY', 'COP', 'CZK', 'HUF', 'IDR', 'INR', 'KRW', 'MXN', 
                 'MYR', 'PEN', 'PHP', 'PLN', 'RON', 'RUB', 'SGD', 'THB', 'TRY', 'TWD', 'ZAR']
for date_str, df in settlement_dates_dict.items():
    df['Currency'] = df['Security'].str.slice(0, 3)
    df['Currency'] = pd.Categorical(df['Currency'], categories=currency_order, ordered=True)
    df.sort_values('Currency', inplace=True)
    df.reset_index(drop=True, inplace=True)

In [432]:
data = {
    'Currency': ['BRL', 'CLP', 'CNY', 'COP', 'CZK', 'HUF', 'IDR', 'INR', 'KRW', 'MXN', 'MYR', 'PEN', 'PHP', 'PLN', 'RON', 'RUB', 'SGD', 'THB', 'TRY', 'TWD', 'UAH', 'ZAR', 'EUR', 'JPY', 'EGP', 'GBP', 'PY'],
    'DayCount': [360, 360, 360, 360, 360, 360, 360, 360, 365, 360, 360, 360, 360, 365, 360, 360, 365, 365, 360, 365, 365, 365, 360, 360, 360, 360, 360],
    'Multiplier': [10000, 1, 10000, 1, 1000, 100, 10000, 100, 1, 10000, 10000, 10000, 10000, 1, 10000, 10000, 100, 10000, 100, 100, 1, 10000, 1, 100, 10000, 10, 10000]
}

convention = pd.DataFrame(data)

In [433]:
reer_dict_final = reer_dict_test.copy()

In [434]:
from xbbg import blp 
import pandas as pd
from dateutil.relativedelta import relativedelta

# Define the list of Bloomberg tickers
tickers = ["BRL BGN Curncy", "CLP BGN Curncy", "CNY CMPN Curncy", "COP REGN Curncy",
           "CZK BGN Curncy", "HUF BGN Curncy", "IDR CMPN Curncy", "MYR CMPN Curncy",
           "MXN BGN Curncy", "PEN BGN Curncy", "PHP CMPN Curncy", "PLN BGN Curncy",
           "RON BGN Curncy", "RUB BGN Curncy", "ZAR BGN Curncy", "THB BGN Curncy",
           "TRY BGN Curncy", "INR CMPN Curncy", "SGD BGN Curncy", "TWD BGN Curncy",
           "KRW CMPN Curncy"]

# Extract currency codes from tickers
currencies = [ticker[:3] for ticker in tickers]

# Get historical data for the past 5 years
end_date = pd.Timestamp.today()
start_date = end_date - relativedelta(years=5)

price_mid = blp.bdh(tickers, 'PX_MID', '20150101', end_date)
price_mid.columns = currencies
price_mid.index = price_mid.index.map(lambda x: x.strftime('%Y%m%d'))


# List of dictionnaries 



momentum_dict # good
volatility_dict # good 
reer_dict_test # good reer_dict_final 
cpi_dict_2# ok
fpa_dict# ok 
tb_dict_with_3m_change # 
data_prices # dataframe of all spots currencies OK 
settlement_dates_dict # all settlement dates dataframe  OK 
data # OK This is fixed 
discount_factors_dict # ok 
inflation 
# code rest of dictionnaries as spot returns, settlement dates, cpi mom/yoy, and rebuild the functions daily (fast) to get final columns Momentum	REER discount	Momentum-adjusted REER discount	3m chg in TB%GDP	Nominal Carry	Real Carry	Implied Volatility Currency	Portfolio Conviction	Portfolio Size	Model Conviction	Model Size

# recreate df for each month Index(['Currency', 'MOV_AVG_5D', 'MOV_AVG_20D', 'Momentum_x', '3M Implied Vol',
       'CPI', 'REER_x', 'REER_5Y_avg_x', 'REER_y', 'REER_5Y_avg_y', 'spot_adj',
       'infla_adj', 'proxy', 'REER_Discount_x', 'REER_Discount_y',
       'Momentum_y', 'Conditional_REER_Discount', 'Symmetrical_REER_Discount',
       'GDP', 'TB_3m_Chg_GDP', 'SPOT_MID', 'FWD_SCALE', 'FPA3M_MID',
       'FPA1M_MID', 'security', 'TB', 'TB_3m_Chg', 'DayCount', 'Multiplier',
       '3m_imp_yield', 'calculated_value', 'CPI YOY', 'Real_Carry',
       'Nominal_carry', 'zscore_RN_part1', 'zscore_RN_part2',
       'zscore_RN_part3', 'zscore_CN_part1', 'zscore_CN_part2',
       'zscore_CN_part3', 'zscore_SN_part1', 'zscore_SN_part2',
       'zscore_SN_part3', 'zscore_RR_part1', 'zscore_RR_part2',
       'zscore_RR_part3', 'zscore_CR_part1', 'zscore_CR_part2',
       'zscore_CR_part3', 'zscore_SR_part1', 'zscore_SR_part2',
       'zscore_SR_part3', 'zscore_RN', 'zscore_CN', 'zscore_SN', 'zscore_RR',
       'zscore_CR', 'zscore_SR'],
      dtype='object')

In [435]:
reer_dict_final = {date_str.replace("-", ""): df for date_str, df in reer_dict_final.items()}


In [436]:
cpi_dict_final = {date_str.replace("-", ""): df for date_str, df in cpi_dict_2.items()}

In [437]:

spot_prices = {}

for date in price_mid.index:
    spot_prices = price_mid.loc[date].dropna() # Drop currencies with NaN values for that date
    spot_prices[date] = pd.DataFrame(spot_prices).transpose()  # Create a DataFrame for each date



In [438]:

spot_prices = {}

for date in price_mid.index:
    spot_priceslocal = price_mid.loc[date]
    spot_prices[date] = pd.DataFrame(spot_priceslocal).transpose()

for date, prices_df in spot_prices.items():
    # Transpose and rename columns
    transposed_df = prices_df.transpose()  # Reset index to make 'index' a column
    transposed_df.columns = ['SPOT_MID']

    spot_prices[date] = transposed_df

In [439]:
from blp import blp
import pandas as pd

def get_daily_discount_factors(ticker, start_date, end_date):
    """Retrieves daily discount factors from Bloomberg.

    Args:
        ticker: Bloomberg ticker (e.g., "USOSFRC Curncy").
        start_date: Start date as a string (YYYY-MM-DD).
        end_date: End date as a string (YYYY-MM-DD).

    Returns:
        A pandas Series with dates as index and discount factors as values, or None if an error occurs.
    """
    try:
        bquery = blp.BlpQuery().start()
        df = bquery.bdh(
            [ticker],
            ["PX_LAST"],
            start_date=start_date,
            end_date=end_date,
            options={"adjustmentSplit": True, "CDR": "5D"},  # Keep CDR option if needed
        )

        # Extract the discount factors and set the date as the index
        discount_factors = df.set_index("date")["PX_LAST"]
        return discount_factors

    except Exception as e:  # Handle potential errors (e.g., connection issues)
        print(f"Error retrieving data: {e}")
        return None

    finally:  # Ensure the Bloomberg connection is stopped
        if bquery:
            bquery.stop()


# Example usage:
start_date = "20240101"
end_date = today.strftime('%Y%m%d')
ticker = "USOSFRC Curncy"

discount_factors = get_daily_discount_factors(ticker, start_date, end_date)

if discount_factors is not None:
    print(discount_factors)

date
2024-01-01    5.3265
2024-01-02    5.3301
2024-01-03    5.3331
2024-01-04    5.3298
2024-01-05    5.3230
               ...  
2024-12-02    4.4512
2024-12-03    4.4469
2024-12-04    4.4329
2024-12-05    4.4273
2024-12-06    4.3938
Name: PX_LAST, Length: 245, dtype: float64


In [440]:
discount_factors_dict = {}

for date, df in discount_factors.items():
    if date not in discount_factors_dict:
        formatted_date = date.strftime('%Y%m%d') #
        discount_factors_dict[formatted_date] = {}  # Initialize inner dictionary
        discount_factors_dict[formatted_date][ticker] = df

In [441]:

merged_dict = {}

for date_str, momentum_df in momentum_dict.items():
    if date_str in volatility_dict:  # Check if the date exists in both dictionaries
        volatility_df = volatility_dict[date_str]

        # Merge the DataFrames on the 'Currency' index
        merged_df = pd.merge(momentum_df, volatility_df, left_index=True, right_index=True, how='inner', suffixes=('_mom', '_vol')) 

        merged_dict[date_str] = merged_df
    else:
        print(f"Warning: Date {date_str} not found in both dictionaries. Skipping.")

In [442]:
final_merged_dict = {}
for date_str, merged_df in merged_dict.items():
    if date_str in reer_dict_final:
        reer_df = reer_dict_final[date_str]
        final_merged_df = pd.merge(reer_df.set_index('Currency'), merged_df, left_index=True, right_index=True, how='inner')
        final_merged_dict[date_str] = final_merged_df
    else: 

        print(f"Warning: Date {date_str} not found in both dictionaries. Skipping.")


In [443]:
final_merged_dict2 = {}
for date_str, merged_df in final_merged_dict.items():
    if date_str in cpi_dict_final:
        cpi_df = cpi_dict_final[date_str]
        final_merged_df = pd.merge(cpi_df.set_index('Currency'), merged_df, left_index=True, right_index=True, how='inner')
        final_merged_dict2[date_str] = final_merged_df
    else: 

        print(f"Warning: Date {date_str} not found in both dictionaries. Skipping.")

In [444]:
final_merged_dict3 = {}
for date_str, merged_df in final_merged_dict2.items():
    if date_str in fpa_dict:
        fpa_df = fpa_dict[date_str]
        final_merged_df = pd.merge(fpa_df, merged_df, left_index=True, right_index=True, how='inner')
        final_merged_dict3[date_str] = final_merged_df
    else: 

        print(f"Warning: Date {date_str} not found in both dictionaries. Skipping.")

In [520]:
final_merged_dict4 = {}
for date_str, merged_df in final_merged_dict3.items():
    if date_str in spot_prices:
        spot_df = spot_prices[date_str]
        final_merged_df = pd.merge(spot_df, merged_df, left_index=True, right_index=True, how='inner')
        final_merged_dict4[date_str] = final_merged_df
    else: 

        print(f"Warning: Date {date_str} not found in both dictionaries. Skipping.")

In [521]:
# merge with trade balance dict 
for date_str, merged_df in final_merged_dict4.items():
    if date_str in tb_dict_with_3m_change:
        tb_df = tb_dict_with_3m_change[date_str]
        final_merged_df = pd.merge(tb_df.set_index('Currency'), merged_df, left_index=True, right_index=True, how='inner')
        final_merged_dict4[date_str] = final_merged_df
    else: 

        print(f"Warning: Date {date_str} not found in both dictionaries. Skipping.")

In [522]:
most_recent_gdp_date = max(GDP_data.keys())
gdp_df = GDP_data[most_recent_gdp_date]

for date_str, merged_df in final_merged_dict4.items():
    if date_str in tb_dict_with_3m_change:
        tb_df = tb_dict_with_3m_change[date_str]

        try: #error handling if column doesn t exist
            gdp_aligned = gdp_df.set_index('Currency').reindex(final_merged_df.index)

            final_merged_df['TB_3m_Chg_GDP'] = final_merged_df['TB_3m_Chg'] / gdp_aligned['GDP']

            final_merged_dict4[date_str] = final_merged_df

        except KeyError as e:
            print(f"Warning: Column {e} not found for date {date_str}. Skipping calculation.")
            # Or log the error for better tracking
    else:
        print(f"Warning: Date {date_str} not found in both dictionaries. Skipping.")


In [523]:
for date_str, df in final_merged_dict4.items():
    df = df.reset_index()
    df = df.drop(['DATE_vol', 'DATE_mom'], axis=1)

    df.rename(columns={'index': 'Currency'}, inplace=True) #Rename the column to 'Currency'
    final_merged_dict4[date_str] = df  # Update the dictionary

In [524]:
import QuantLib as ql
data = {
    'Currency': ['BRL', 'CLP', 'CNY', 'COP', 'CZK', 'HUF', 'IDR', 'INR', 'KRW', 'MXN', 'MYR', 'PEN', 'PHP', 'PLN', 'RON', 'RUB', 'SGD', 'THB', 'TRY', 'TWD', 'UAH', 'ZAR', 'EUR', 'JPY', 'EGP', 'GBP', 'PY'],
    'DayCount': [360, 360, 360, 360, 360, 360, 360, 360, 365, 360, 360, 360, 360, 365, 360, 360, 365, 365, 360, 365, 365, 365, 360, 360, 360, 360, 360],
    'Multiplier': [10000, 1, 10000, 1, 1000, 100, 10000, 100, 1, 10000, 10000, 10000, 10000, 1, 10000, 10000, 100, 10000, 100, 100, 1, 10000, 1, 100, 10000, 10, 10000]
}
###########  GOOD  3 m implied yield##################

convention = pd.DataFrame(data)
def calculate_value(df, date_str):  # Add date_str as an argument
    """Calculates the value based on the provided formula, adapted for dictionary access.

    Args:
        df: The DataFrame for a specific date.
        date_str: The date string key for accessing settlement_dates_dict.

    Returns:
        A Series containing the calculated values, or None if an error occurs.
    """
    day_count = convention['DayCount']
    try:
        # Vectorized approach
        settlement_dates = settlement_dates_dict[date_str]
        settlement_dates = settlement_dates.drop_duplicates(subset='Currency', keep='first')

        settlement_dates = settlement_dates.set_index('Currency')['Settlement_Date']
        settlement_dates_ql = settlement_dates.apply(lambda x: ql.Date(x, '%Y-%m-%d'))
        date_ql = ql.Date(date_str, '%Y%m%d')
        total_days = settlement_dates_ql - date_ql
        total_days = total_days.reindex(final_merged_dict4[date_str]['Currency'])

        total_days = total_days.reset_index(drop=True)

        frd_pts_ask = final_merged_dict4[date_str]["FPA3M_MID"]
        spot = df['SPOT_MID']
        discount_factor = discount_factors_dict[date_str]['USOSFRC Curncy']
    


        # Vectorized Calculation:
        value = 100 * (
            (1 + frd_pts_ask / spot)
            * (1 + discount_factor / 100 * total_days / day_count)
            - 1
        ) * day_count / total_days
        return value  # Return the calculated Series

    except Exception as e:
        print(f"Error calculating values for {date_str}: {e}")
        return None




for date_str, df in final_merged_dict4.items():
    #print(final_merged_dict4[date_str]['FPA3M_MID'])
    # Apply the function
    try:
        df['3m_imp_yield'] = calculate_value(df, date_str)
        
    except Exception as e: # Handle cases where the date might be missing or data is inconsistent
        print(f"Error processing date {date_str}. Skipping. Error: {e}")
        continue




In [525]:
from xbbg import blp

def calculate_real_carry(df, date_str, inflation): # Add date_str and inflation arguments
    """Calculates real carry, adapted for dictionary and vectorized operations.

    Args:
        df: DataFrame for the specific date.
        date_str: Not used in this function (kept for consistency with other functions).
        inflation: The inflation DataFrame.

    Returns:
        A Series with calculated real carry values, or None if an error occurs.
    """
    try:
        imp_yield = df['3m_imp_yield']
        discount_factor = discount_factors_dict[date_str]['USOSFRC Curncy'] # Access from dictionary
        
        # Merge inflation data with df based on 'Currency'
        df = pd.merge(df, inflation, on='Currency', how='left')  # Left merge to keep all currencies in df
        cpi_emergent = df['CPI YOY']  # Access CPI from the merged DataFrame


        cpi_us = blp.bdp(['CPI YOY Index'], 'last_price').values[0][0]


        real_carry = ((1 + imp_yield / 100) / (1 + cpi_emergent / 100)
                     - (1 + discount_factor / 100) / (1 + cpi_us / 100))
        return real_carry

    except Exception as e:
        print(f"Error calculating real carry for {date_str}: {e}")
        return None



for date_str, df in final_merged_dict4.items():
    try:

        df['Real_Carry'] = calculate_real_carry(df, date_str, inflation)
    except Exception as e:
        print(f"Error processing date {date_str}: {e}")
        continue  # Or handle the error as needed

In [526]:
def calculate_nominal_carry(df, date_str):  # Add date_str argument
    """Calculates nominal carry, adapted for dictionary access and vectorized operations.

    Args:
        df: DataFrame for the specific date.
        date_str: The date string for accessing discount factors.

    Returns:
        A Series of calculated nominal carry values or None if an error occurs.
    """

    try:
        imp_yield = df['3m_imp_yield']
        discount_factor = discount_factors_dict[date_str]['USOSFRC Curncy'] # Access from dict

        # Vectorized Calculation
        nominal_carry = ((1 + imp_yield / 100) / (1 + discount_factor / 100) - 1)
        return nominal_carry

    except Exception as e:
        print(f"Error calculating nominal carry for {date_str}: {e}")
        return None




for date_str, df in final_merged_dict4.items():
    try:
        df['Nominal_carry'] = calculate_nominal_carry(df, date_str)

    except Exception as e:  # Handle potential errors (missing data, type errors)
        print(f"Error processing date {date_str}: {e}")
        continue

In [527]:
def calculate_symmetrical_reer(df):
    """Calculates Symmetrical REER Discount based on the Excel logic."""
    if 'REER_Discount' not in df.columns:
        print(f"Warning: 'REER_Discount' column not found for date {date}. Skipping calculation.")  #or logging
        return df  

    df['Symmetrical_REER_Discount'] = df.apply(
        lambda row: row['REER_Discount'] * (
            1 if (row['Momentum'] == 1 and row['REER_Discount'] < 0) or (row['Momentum'] == -1 and row['REER_Discount'] > 0)
            else 0
        ),
        axis=1
    )

    
    # Calculate average standard deviation for currencies (excluding 'Avg/Std' row)
    #currencies_df = df[df['Currency'] != 'Avg/Std']
    #avg_std = currencies_df['Symmetrical_REER_Discount'].std()

    # Update 'Avg/Std' row
    #df.loc[df['Currency'] == 'Avg/Std', 'Symmetrical_REER_Discount'] = avg_std

    return df

for date, df in final_merged_dict4.items():
    final_merged_dict4[date] = calculate_symmetrical_reer(df)

In [528]:
def add_conditional_reer(df):
    """
    Adds 'Conditional_REER_Discount' column if it doesn't exist.
    """
    if 'REER_Discount' not in df.columns:
        print(f"Warning: 'REER_Discount' column not found for date {date}. Skipping calculation.")  #or logging
        return df
    if 'Conditional_REER_Discount' not in df.columns:
        df['Conditional_REER_Discount'] = df.apply(
            lambda row: row['Momentum'] * row['REER_Discount'] * (row['REER_Discount'] < 0) if row['Momentum'] == 1 else 0, 
            axis=1
        )
    return df


# Apply the function to each DataFrame in the dictionary
for date, df in final_merged_dict4.items():
    final_merged_dict4[date] = add_conditional_reer(df)

In [ ]:
### ADAPTED Z SCORE FUNCTION #######
models = {  # Dictionary mapping short model names to criteria lists
    "RN": ["REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "CN": ["Conditional_REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "SN": ["Symmetrical_REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "RR": ["REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
    "CR": ["Conditional_REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
    "SR": ["Symmetrical_REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
}
def calculate_zscore(df, criteria):
    """Calculates the z-score based on the provided criteria."""
    zscore_parts = []

    for i, criterion in enumerate(criteria):
        if df[criterion].std() == 0:  # Handle zero standard deviation
            zscore_part = 0
        elif criterion in ["Symmetrical_REER_Discount", "Conditional_REER_Discount"]:
            zscore_part = -df[criterion] / df[criterion].std()
        else:
            zscore_part = df[criterion] / df[criterion].std()

        zscore_parts.append(zscore_part)

    # Return NA for the 'Avg/Std' row
    return np.where(df['Currency'] == 'Avg/Std', np.nan, sum(zscore_parts)/3)


# Loop through each date in the dictionary
for date, df in final_merged_dict4.items():
    print(date)
    # Find the recommended model for the current date
    model_row = model_list[model_list['Date'] == date]
    
    if not model_row.empty:
        model_name = model_row['model'].iloc[0][:2]  # Extract first two characters (e.g., "SN", "RN")
        criteria = models[model_name]  # Get criteria for the model


        df['z_score'] = calculate_zscore(df, criteria)



In [ ]:
import pandas as pd
import numpy as np
models = {
    "RN": ["REER_Discount_x", "TB_3m_Chg_GDP", "Nominal_carry"],
    "CN": ["Conditional_REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "SN": ["Symmetrical_REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "RR": ["REER_Discount_x", "TB_3m_Chg_GDP", "Real_Carry"],
    "CR": ["Conditional_REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
    "SR": ["Symmetrical_REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
}

def round_2x_to_half(x):
    two_x = 2 * x
    return round(two_x * 2) / 2 if two_x >= 0 else -np.round(-two_x * 2) / 2

# Conviction mapping dictionaries (define outside the function)
conviction_mapping_agnostic = {  # No change here
    0: 3, 1: 3, 2: 2, 3: 2, 4: 2, 5: 1, 6: 1, 7: 1,
    8: 0, 9: 0, 10: 0, 11: 0, 12: -1, 13: -1, 14: -1,
    15: -2, 16: -2, 17: -2, 18: -3, 19: -3
}


def calculate_conviction_and_zscore(df, model_name):
    """Calculates both conviction and z-score, handling 'Avg/Std' row and NaNs."""

    model_type = model_name.split("-")[1]
    short_model_name = model_name[:2]
    criteria = models.get(short_model_name)

    if criteria:
        # Filter out "Avg/Std" *before* calculations:
        df_currencies = df[df['Currency'] != 'Avg/Std'].copy()  #Important: create a copy to avoid SettingWithCopyWarning



        # 1. Calculate Z-score:
        df_currencies[f'zscore_{short_model_name}'] = calculate_zscore(df_currencies, criteria)

        # 2. Calculate Rank (and fill NaNs if any):
        df_currencies[f'rank_{short_model_name}'] = df_currencies[f'zscore_{short_model_name}'].rank(ascending=False, method='dense').fillna(-1).astype(int)


        # 3. Calculate Conviction:
        if model_type == 'Agn':
            df_sorted = df_currencies.sort_values(by=f'rank_{short_model_name}')

            df_sorted[f'conviction_{model_name}'] = df_sorted[f'rank_{short_model_name}'].apply(
                 lambda rank: conviction_mapping_agnostic.get(int(rank) - 1, np.nan) if not pd.isna(rank) else None)

            df = df.merge(df_sorted[[f'conviction_{model_name}', 'Currency']], on='Currency', how='left') #Important: add currency to merge on

        elif model_type == 'Awa':

            df_currencies[f'conviction_{model_name}'] = df_currencies.apply(
                lambda row: _calculate_awa_conviction(row, short_model_name), axis=1
             )
            df = df.merge(df_currencies[[f'conviction_{model_name}', 'Currency']], on='Currency', how='left')#Merge the result back


    return df

def _calculate_awa_conviction(row, model_name): #Correct the ranges 
    try:
        rank = int(row[f'rank_{model_name}'])
    except (ValueError, TypeError):  #Catching more types of errors
        return np.nan  # Or another default value as needed


    zscore = row[f'zscore_{model_name}']
    if rank <= 8:
        conviction = max(-3, min(3, np.round(zscore * 4)/2 ))
    elif rank <=12: #Between 9 and 12
        conviction=0
    elif rank >= 13 :# >= 13

        conviction = max(-3, min(3, round_2x_to_half(zscore)))
    else:
        return np.nan
    return conviction 




# Main loop:
# Ensure 'Date' in model_list is datetime64[ns] type:
model_list['Date'] = model_list['Date'].dt.strftime('%Y-%m-%d')

for date_str, df in date_metric_dictionary.items():
    #Ensure also that your dictionnary keys are in datetime format, without time 
    date = pd.to_datetime(date_str).strftime('%Y-%m-%d')#Remove time from timestamp
    
    model_row = model_list[model_list['Date'] == date]

    if not model_row.empty:

        model_name = model_row['model'].iloc[0]
        date_metric_dictionary[date_str] = calculate_conviction_and_zscore(df.copy(), model_name) 

    else:
        print(f"No model found for {date}.")


In [531]:
final_merged_dict4['20241206']

,Currency,TB,TB_3m_Chg,SPOT_MID,FPA3M_MID,CPI,REER,REER_5Y_avg,spot_adj,infla_adj,...,MOV_AVG_5D,MOV_AVG_20D,Momentum,3M Implied Vol,TB_3m_Chg_GDP,3m_imp_yield,Real_Carry,Nominal_carry,Symmetrical_REER_Discount,Conditional_REER_Discount
0,BRL,79.20,-10.9,6.07980,0.075493,0.44,103.66,106.868361,-0.029861,-0.000357,...,6.0503,5.8736,1,13.8900,-0.005015,9.203825,0.024936,0.046076,-0.059051,-0.059051
1,CLP,21.50,1.8,975.45000,1.2,0.1,102.22,102.706557,0.006366,0.000407,...,974.2300,976.5800,-1,13.1000,0.005365,4.870346,-0.015856,0.004565,0.001677,0.000000
2,CNY,929.10,46.5,7.27170,-0.031829,0.0,92.01,98.39377,-0.009856,-0.000013,...,7.2710,7.2468,1,6.2800,0.002613,2.623753,0.005685,-0.016955,-0.074098,-0.074098
3,COP,-9.80,-0.2,4404.12500,51.137,0.24,104.64,99.188197,0.022036,0.001615,...,4428.5800,4418.3800,1,13.0925,-0.000550,8.891644,0.015546,0.043085,0.000000,0.000000
4,CZK,9.80,0.4,23.77055,-0.04859,-0.4,119.03,111.737541,0.002319,-0.003515,...,23.8813,23.9580,-1,8.4950,0.001209,3.601962,-0.009682,-0.007585,0.067007,0.000000
5,HUF,12.40,-0.6,392.34100,1.7,-0.1,106.12,102.799016,-0.020019,-0.000306,...,393.0500,389.7800,1,11.3925,-0.002825,6.072272,0.010349,0.016078,0.000000,0.000000
6,IDR,30.10,-1.2,15853.00000,57.0,-0.12,99.73,100.533115,0.003630,-0.001191,...,15896.0000,15848.0000,1,7.2000,-0.000875,5.786611,0.022597,0.013342,-0.004617,-0.004617
7,MYR,27.50,-2.2,4.42200,-0.01152,0.0,101.11,97.83918,0.002151,-0.000175,...,4.4454,4.4545,-1,6.4250,-0.005505,3.384634,-0.002914,-0.009667,0.035618,0.000000
8,MXN,-5.80,1.7,20.23115,0.303254,0.05,121.26,115.177541,-0.003822,-0.000356,...,20.2920,20.3733,-1,13.2080,0.000950,10.280046,0.035209,0.056385,0.048714,0.000000
9,PEN,2342.87,337.8,3.72123,0.0055,-0.12,102.27,97.508525,0.033442,-0.001969,...,3.7358,3.7712,-1,6.9100,1.262318,4.966339,0.011497,0.005484,0.083492,0.000000
